In [3]:
import sys
sys.path.append('../scripts/')
from read_utils import read_file, create_folder, temp_record_query, temp_record_sdf

In [4]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [5]:
spark = (
    # Create a spark session (which will run spark jobs)
    SparkSession.builder.appName("Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config('spark.executor.memory','10g')
    .config('spark.driver.memory','12g')
    .config('spark.driver.maxResultsSize', '10GiB')
    # .config("spark.network.timeout", "3600s")
    # .master("local[6]")
    .getOrCreate()
    )

22/10/05 00:46:47 WARN Utils: Your hostname, Runyus-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.3.12 instead (on interface en0)
22/10/05 00:46:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/05 00:46:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/05 00:46:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
trans_sdf = read_file(spark, 'changed_data', '../data/curated/')

|> Loading File...


|> Loading Finished!


-RECORD 0--------------------------------------------------------------------------
 user_id           | 8972                                                          
 merchant_abn      | 59658193201                                                   
 dollar_value      | 182.16129389048888                                            
 order_id          | 5a2112b5-6a59-4ceb-95a5-655fa5915c7a                          
 merchant_name     | Id Ante Institute                                             
 tags              | cable, satellite, and other pay television and radio services 
 take_rate         | 6.16                                                          
 type              | a                                                             
 postcode          | 5311                                                          
 consumer_id       | 1115127                                                       
 name              | Mr. Bradley Moore                                      

In [7]:
consumer_fraud_sdf = read_file(spark,'consumer_fraud_probability.csv', '../data/tables/', 'csv')

|> Loading File...
|> Loading Finished!
-RECORD 0-----------------------------
 user_id           | 6228             
 order_datetime    | 2021-12-19       
 fraud_probability | 97.6298077657765 
only showing top 1 row



In [8]:
consumer_fraud_sdf.show(5)

+-------+--------------+-----------------+
|user_id|order_datetime|fraud_probability|
+-------+--------------+-----------------+
|   6228|    2021-12-19| 97.6298077657765|
|  21419|    2021-12-10|99.24738020302328|
|   5606|    2021-10-17|84.05825045251777|
|   3101|    2021-04-17|91.42192091901347|
|  22239|    2021-10-19|94.70342477508035|
+-------+--------------+-----------------+
only showing top 5 rows



In [9]:
trans_sdf.show(5)

+-------+------------+------------------+--------------------+--------------------+--------------------+---------+----+--------+-----------+-----------------+--------------------+-----+------+----------+-----------------+---------------+----------+------------+-----------------+-----------+---------+--------+---------------+--------------+
|user_id|merchant_abn|      dollar_value|            order_id|       merchant_name|                tags|take_rate|type|postcode|consumer_id|             name|             address|state|gender|POA_CODE21|       population|New cases / day|     month|total_retail|__index_level_0__|genderIndex|typeIndex|tagIndex|        tag_vec|order_datetime|
+-------+------------+------------------+--------------------+--------------------+--------------------+---------+----+--------+-----------+-----------------+--------------------+-----+------+----------+-----------------+---------------+----------+------------+-----------------+-----------+---------+--------+------

In [10]:
sdf = trans_sdf.select( 'user_id', 'merchant_abn', 'order_datetime', 'dollar_value', 'take_rate', 'consumer_id', 'POA_CODE21', 'genderIndex', 'typeIndex', 'tagIndex', 'tag_vec')

In [11]:
sdf.show(5)

+-------+------------+--------------+------------------+---------+-----------+----------+-----------+---------+--------+---------------+
|user_id|merchant_abn|order_datetime|      dollar_value|take_rate|consumer_id|POA_CODE21|genderIndex|typeIndex|tagIndex|        tag_vec|
+-------+------------+--------------+------------------+---------+-----------+----------+-----------+---------+--------+---------------+
|   8972| 59658193201|    2021-11-26|182.16129389048888|     6.16|    1115127|      5311|        0.0|      0.0|     4.0| (24,[4],[1.0])|
|   9044| 83620670827|    2021-11-26| 93.18042034505228|     3.43|     916302|      3522|        1.0|      1.0|     4.0| (24,[4],[1.0])|
|   8968| 46887242827|    2021-11-26| 2132.463971960699|     2.43|     157590|      3483|        0.0|      2.0|    21.0|(24,[21],[1.0])|
|   8972| 86578477987|    2021-11-26| 75.91366457857609|     6.43|    1115127|      5311|        0.0|      0.0|     3.0| (24,[3],[1.0])|
|   8976| 15299889494|    2021-11-26|15.3

In [16]:
import pyspark.sql.functions as F
sdf_daily = sdf.groupby('user_id', 'consumer_id', 'order_datetime', 'take_rate', 'tagIndex', 'tag_vec').agg(
    F.count("*").alias("count"), 
    F.avg("dollar_value").alias("dollar_average"),
    F.min("dollar_value").alias("dollar_min"),
    F.max("dollar_value").alias("dollar_max"),
    )

In [17]:
sdf_daily

user_id,consumer_id,order_datetime,take_rate,tagIndex,tag_vec,count,dollar_average,dollar_min,dollar_max
9748,717893,2021-11-26,6.22,0.0,"(24,[0],[1.0])",1,24.843222036146003,24.843222036146003,24.843222036146003
7209,568273,2021-11-26,1.19,20.0,"(24,[20],[1.0])",1,299.87582108713747,299.87582108713747,299.87582108713747
12046,24783,2021-11-26,5.71,11.0,"(24,[11],[1.0])",1,617.8117601308935,617.8117601308935,617.8117601308935
12177,1119876,2021-11-26,6.35,5.0,"(24,[5],[1.0])",1,26.81515731654183,26.81515731654183,26.81515731654183
12495,948299,2021-11-26,4.08,2.0,"(24,[2],[1.0])",1,80.84890806887542,80.84890806887542,80.84890806887542
10147,561869,2021-11-26,4.65,10.0,"(24,[10],[1.0])",1,358.08361920798586,358.08361920798586,358.08361920798586
11415,1317706,2021-11-26,5.91,4.0,"(24,[4],[1.0])",1,160.2180858828281,160.2180858828281,160.2180858828281
10838,34350,2021-11-26,4.35,0.0,"(24,[0],[1.0])",1,52.8875591766947,52.8875591766947,52.8875591766947
10791,1459924,2021-11-26,6.05,10.0,"(24,[10],[1.0])",1,138.38129493767156,138.38129493767156,138.38129493767156
2893,596153,2021-11-26,6.01,0.0,"(24,[0],[1.0])",1,290.4044762772601,290.4044762772601,290.4044762772601


In [18]:
consumer_fraud_sdf.show

<bound method DataFrame.show of +-------+--------------+-----------------+
|user_id|order_datetime|fraud_probability|
+-------+--------------+-----------------+
|   6228|    2021-12-19| 97.6298077657765|
|  21419|    2021-12-10|99.24738020302328|
|   5606|    2021-10-17|84.05825045251777|
|   3101|    2021-04-17|91.42192091901347|
|  22239|    2021-10-19|94.70342477508035|
|  16556|    2022-02-20|89.65663294494827|
|  10278|    2021-09-28|83.59136689427714|
|  15790|    2021-12-30|71.77065889280253|
|   5233|    2021-08-29|85.87123303878818|
|    230|    2021-08-28|86.28328808934151|
|  13601|    2021-12-26|83.13696487489679|
|   6383|    2021-09-15| 66.2676451623754|
|   3513|    2022-02-27|75.16981192247916|
|  18658|    2021-10-19|82.98609082999361|
|   5965|    2021-11-14|69.37164467869053|
|  18714|    2021-11-14|83.78813794627237|
|  22957|    2022-02-12|82.79065699075498|
|  20118|    2021-09-05|80.34030486265003|
|   6436|    2021-12-24|84.81618344606828|
|  17900|    2022-02-2

In [19]:
train_data = sdf_daily.join(consumer_fraud_sdf, on=['user_id', 'order_datetime'], how = 'inner')

In [20]:
trans_data = sdf_daily.join(consumer_fraud_sdf, on=['user_id', 'order_datetime'], how = 'left_outer')

In [21]:
train_data

user_id,order_datetime,consumer_id,take_rate,tagIndex,tag_vec,count,dollar_average,dollar_min,dollar_max,fraud_probability
11415,2021-11-26,1317706,5.91,4.0,"(24,[4],[1.0])",1,160.2180858828281,160.2180858828281,160.2180858828281,10.595743221893015
7602,2021-11-26,124099,4.42,20.0,"(24,[20],[1.0])",1,2082.9234439997836,2082.9234439997836,2082.9234439997836,8.327274200093298
15976,2021-11-26,234865,3.8,1.0,"(24,[1],[1.0])",1,65.56040163447034,65.56040163447034,65.56040163447034,8.925014091169167
21493,2021-11-26,1084506,6.31,14.0,"(24,[14],[1.0])",1,58.436673163850905,58.436673163850905,58.436673163850905,15.940146016029113
15766,2021-11-26,1259965,6.61,1.0,"(24,[1],[1.0])",1,81.43860616644918,81.43860616644918,81.43860616644918,8.856425404104968
8125,2021-11-26,875806,3.72,11.0,"(24,[11],[1.0])",1,158.77637277548067,158.77637277548067,158.77637277548067,9.292232631765675
19062,2021-11-26,88353,6.39,0.0,"(24,[0],[1.0])",1,20.415018046326782,20.415018046326782,20.415018046326782,9.901323164114215
7096,2021-11-26,1290631,2.45,22.0,"(24,[22],[1.0])",1,2234.0604728575718,2234.0604728575718,2234.0604728575718,11.335927429622716
17150,2021-11-26,774302,6.48,9.0,"(24,[9],[1.0])",1,6.166761011977075,6.166761011977075,6.166761011977075,11.920239170070989
11432,2021-11-26,751988,6.11,2.0,"(24,[2],[1.0])",1,16.02183412132618,16.02183412132618,16.02183412132618,16.516447904450796


In [22]:
trans_data

user_id,order_datetime,consumer_id,take_rate,tagIndex,tag_vec,count,dollar_average,dollar_min,dollar_max,fraud_probability
9748,2021-11-26,717893,6.22,0.0,"(24,[0],[1.0])",1,24.843222036146003,24.843222036146003,24.843222036146003,null
7209,2021-11-26,568273,1.19,20.0,"(24,[20],[1.0])",1,299.87582108713747,299.87582108713747,299.87582108713747,null
12046,2021-11-26,24783,5.71,11.0,"(24,[11],[1.0])",1,617.8117601308935,617.8117601308935,617.8117601308935,null
12177,2021-11-26,1119876,6.35,5.0,"(24,[5],[1.0])",1,26.81515731654183,26.81515731654183,26.81515731654183,null
12495,2021-11-26,948299,4.08,2.0,"(24,[2],[1.0])",1,80.84890806887542,80.84890806887542,80.84890806887542,null
10147,2021-11-26,561869,4.65,10.0,"(24,[10],[1.0])",1,358.08361920798586,358.08361920798586,358.08361920798586,null
11415,2021-11-26,1317706,5.91,4.0,"(24,[4],[1.0])",1,160.2180858828281,160.2180858828281,160.2180858828281,10.595743221893015
10838,2021-11-26,34350,4.35,0.0,"(24,[0],[1.0])",1,52.8875591766947,52.8875591766947,52.8875591766947,null
10791,2021-11-26,1459924,6.05,10.0,"(24,[10],[1.0])",1,138.38129493767156,138.38129493767156,138.38129493767156,null
2893,2021-11-26,596153,6.01,0.0,"(24,[0],[1.0])",1,290.4044762772601,290.4044762772601,290.4044762772601,null


In [23]:
from pyspark.sql.functions import year, month, dayofmonth

In [24]:
train_fraud_table = (train_data
               .withColumn('order_year', year(train_data.order_datetime))
               .withColumn('order_month', month(train_data.order_datetime))
               .withColumn('order_day', dayofmonth(train_data.order_datetime))
               )

In [26]:
train_fraud_table = train_fraud_table.drop('order_datetime')

In [27]:
train_fraud_table.show(5)

+-------+-----------+---------+--------+---------------+-----+------------------+------------------+------------------+------------------+----------+-----------+---------+
|user_id|consumer_id|take_rate|tagIndex|        tag_vec|count|    dollar_average|        dollar_min|        dollar_max| fraud_probability|order_year|order_month|order_day|
+-------+-----------+---------+--------+---------------+-----+------------------+------------------+------------------+------------------+----------+-----------+---------+
|  11415|    1317706|     5.91|     4.0| (24,[4],[1.0])|    1| 160.2180858828281| 160.2180858828281| 160.2180858828281|10.595743221893015|      2021|         11|       26|
|   7602|     124099|     4.42|    20.0|(24,[20],[1.0])|    1|2082.9234439997836|2082.9234439997836|2082.9234439997836| 8.327274200093298|      2021|         11|       26|
|  15976|     234865|      3.8|     1.0| (24,[1],[1.0])|    1| 65.56040163447034| 65.56040163447034| 65.56040163447034| 8.925014091169167|  

In [28]:
train_fraud_table = (train_fraud_table
                     .withColumn('fraud_probability', train_fraud_table['fraud_probability'].cast('double'))
                     .withColumn('consumer_id',train_fraud_table['consumer_id'].cast('double'))
                     )

- Test Data

In [29]:
test_fraud_table = (trans_data
               .withColumn('order_year', year(train_data.order_datetime))
               .withColumn('order_month', month(train_data.order_datetime))
               .withColumn('order_day', dayofmonth(train_data.order_datetime))
               )

In [30]:
train_fraud_table = (train_fraud_table
                     .withColumn('fraud_probability', train_fraud_table['fraud_probability'].cast('double'))
                     .withColumn('consumer_id',train_fraud_table['consumer_id'].cast('double'))
                     )

In [31]:
test_fraud_table = test_fraud_table.drop('order_datetime','fraud_probability')

In [32]:
test_fraud_table.show(5)

+-------+-----------+---------+--------+---------------+-----+------------------+------------------+------------------+----------+-----------+---------+
|user_id|consumer_id|take_rate|tagIndex|        tag_vec|count|    dollar_average|        dollar_min|        dollar_max|order_year|order_month|order_day|
+-------+-----------+---------+--------+---------------+-----+------------------+------------------+------------------+----------+-----------+---------+
|   9748|     717893|     6.22|     0.0| (24,[0],[1.0])|    1|24.843222036146003|24.843222036146003|24.843222036146003|      2021|         11|       26|
|   7209|     568273|     1.19|    20.0|(24,[20],[1.0])|    1|299.87582108713747|299.87582108713747|299.87582108713747|      2021|         11|       26|
|  12046|      24783|     5.71|    11.0|(24,[11],[1.0])|    1| 617.8117601308935| 617.8117601308935| 617.8117601308935|      2021|         11|       26|
|  12177|    1119876|     6.35|     5.0| (24,[5],[1.0])|    1| 26.81515731654183| 

- Model

In [35]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [36]:
input_col = ['user_id', 'take_rate', 'tagIndex', 'tag_vec', 'count', 'dollar_average', 'dollar_min', 'dollar_max', 'order_year', 'order_month', 'order_day']

In [37]:
vecAssembler = VectorAssembler(inputCols=input_col, outputCol="features")

In [38]:
train_dataset = vecAssembler.transform(train_fraud_table)

In [39]:
train_dataset.show(5)

+-------+-----------+---------+--------+---------------+-----+------------------+------------------+------------------+------------------+----------+-----------+---------+--------------------+
|user_id|consumer_id|take_rate|tagIndex|        tag_vec|count|    dollar_average|        dollar_min|        dollar_max| fraud_probability|order_year|order_month|order_day|            features|
+-------+-----------+---------+--------+---------------+-----+------------------+------------------+------------------+------------------+----------+-----------+---------+--------------------+
|  11415|  1317706.0|     5.91|     4.0| (24,[4],[1.0])|    1| 160.2180858828281| 160.2180858828281| 160.2180858828281|10.595743221893015|      2021|         11|       26|(34,[0,1,2,7,27,2...|
|   7602|   124099.0|     4.42|    20.0|(24,[20],[1.0])|    1|2082.9234439997836|2082.9234439997836|2082.9234439997836| 8.327274200093298|      2021|         11|       26|(34,[0,1,2,23,27,...|
|  15976|   234865.0|      3.8|    

In [40]:
log_reg = LinearRegression(labelCol = 'fraud_probability').fit(train_dataset)

22/10/05 00:59:26 WARN Instrumentation: [89978376] regParam is zero, which might cause numerical instability and overfitting.


22/10/05 00:59:29 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/10/05 00:59:29 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


22/10/05 00:59:30 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK
22/10/05 00:59:30 WARN Instrumentation: [89978376] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


In [41]:
test_feature = vecAssembler.transform(test_fraud_table)

In [42]:
test_feature.show(5)

+-------+-----------+---------+--------+---------------+-----+------------------+------------------+------------------+----------+-----------+---------+--------------------+
|user_id|consumer_id|take_rate|tagIndex|        tag_vec|count|    dollar_average|        dollar_min|        dollar_max|order_year|order_month|order_day|            features|
+-------+-----------+---------+--------+---------------+-----+------------------+------------------+------------------+----------+-----------+---------+--------------------+
|   9748|     717893|     6.22|     0.0| (24,[0],[1.0])|    1|24.843222036146003|24.843222036146003|24.843222036146003|      2021|         11|       26|(34,[0,1,3,27,28,...|
|   7209|     568273|     1.19|    20.0|(24,[20],[1.0])|    1|299.87582108713747|299.87582108713747|299.87582108713747|      2021|         11|       26|(34,[0,1,2,23,27,...|
|  12046|      24783|     5.71|    11.0|(24,[11],[1.0])|    1| 617.8117601308935| 617.8117601308935| 617.8117601308935|      2021|

In [43]:
test_pred = log_reg.transform(test_feature)

In [44]:
test_pred.show(5)

+-------+-----------+---------+--------+---------------+-----+------------------+------------------+------------------+----------+-----------+---------+--------------------+------------------+
|user_id|consumer_id|take_rate|tagIndex|        tag_vec|count|    dollar_average|        dollar_min|        dollar_max|order_year|order_month|order_day|            features|        prediction|
+-------+-----------+---------+--------+---------------+-----+------------------+------------------+------------------+----------+-----------+---------+--------------------+------------------+
|   9748|     717893|     6.22|     0.0| (24,[0],[1.0])|    1|24.843222036146003|24.843222036146003|24.843222036146003|      2021|         11|       26|(34,[0,1,3,27,28,...|11.511066708179214|
|   7209|     568273|     1.19|    20.0|(24,[20],[1.0])|    1|299.87582108713747|299.87582108713747|299.87582108713747|      2021|         11|       26|(34,[0,1,2,23,27,...|11.567064332976543|
|  12046|      24783|     5.71|    

In [45]:
test_pred = test_pred.withColumnRenamed('prediction', 'fraud_probability')

In [46]:
test_pred.show(5)

+-------+-----------+---------+--------+---------------+-----+------------------+------------------+------------------+----------+-----------+---------+--------------------+------------------+
|user_id|consumer_id|take_rate|tagIndex|        tag_vec|count|    dollar_average|        dollar_min|        dollar_max|order_year|order_month|order_day|            features| fraud_probability|
+-------+-----------+---------+--------+---------------+-----+------------------+------------------+------------------+----------+-----------+---------+--------------------+------------------+
|   9748|     717893|     6.22|     0.0| (24,[0],[1.0])|    1|24.843222036146003|24.843222036146003|24.843222036146003|      2021|         11|       26|(34,[0,1,3,27,28,...|11.511066708179214|
|   7209|     568273|     1.19|    20.0|(24,[20],[1.0])|    1|299.87582108713747|299.87582108713747|299.87582108713747|      2021|         11|       26|(34,[0,1,2,23,27,...|11.567064332976543|
|  12046|      24783|     5.71|    

In [47]:
temp_record_sdf(test_pred, '../data/curated/consumer_fraud.parquet')

|> Waitting for saving...


|> Save Successfully!
